Рассмотрим задачу о намерзании льда на поверхности водоёма глубиной $1$ метр, который в начальный момент времени находится полностью в жидком состоянии.

Для этого решим уравнение теплопроводности, при учёте фазового перехода в среде, считая, что плотность среды $\rho$ не изменяется при фазовом перходе.

$$\begin{cases}
    \rho c_s \frac{\partial T}{\partial t} = a_s \frac{\partial^2 T}{\partial x^2}, \; 0 < x < y(t) \\
    \rho c_f \frac{\partial T}{\partial t} = a_f \frac{\partial^2 T}{\partial x^2}, \; y(t) < x < 1 \\
    T(y(t), t) = T_p
\end{cases}$$

Здесь $y(t)$ - граница раздела фаз, $T(x, t)$ - распределние температуры, $T_p = 273 \text{К}$ - температура плавления льда.

Также необходимо учесть условие баланса энергии при движении фазового фронта:

$$a_s \frac{\partial T}{\partial x}(y(t) - 0) - a_f \frac{\partial T}{\partial x}(y(t) + 0) = \rho L \frac{\partial y}{\partial t}$$

В качестве начально-краевых условий возмём:

$T(x, 0) = \varphi(x) = \left( 7x + 273 \right) \text{К}$

$T(0, t) = 273 [1 - 13(1 - e^{-10t})] \text{К}$ - поверхность водоёма

$T(1, t) = 280 \text{К}$ - дно водоёма

Схема Лакса-Вендроффа

In [ ]:
from PDElib import *

@dataclass
class HEParams:
    rho : float # density
    T_p : float # phase change temperature
    L : float # heat of fusion
    # fluid:
    c_f : float # heat capacity
    a_f : float # thermal conductivity
    #solid:
    c_s : float # heat capacity
    a_s : float # thermal conductivity

# Heat Equation Solver
# solve rho*c*T't = a*T''x = 0
# c = c_s if x < y(t) else c = c_f
# a = a_s if x < y(t) else a = a_f
# y(t) defined by rule T(y(t), t) = T_p
# WARNING: The temperature should increase monotonically from left to right!
# takes into account the balance of energies:
# a_s*T'x(y(t) - 0) - a_f*T'x(y(t) + 0) = rho*L*y't
# T(x, 0) = phi(1)
# T(0, t) = psi_l(t)
# T(l, t) = psi_r(t)
# returns [y[t], u[t][x]]
class HESolver:
    @staticmethod
    def __prepare(grid : Grid, phi : RtoR, psi_l : RtoR, psi_r : RtoR):
        tau = grid.T / grid.N
        h = grid.L / grid.M
        u = [[phi(m*h) for m in range(grid.M + 1)]]
        for n in range(1, grid.N + 1):
            u += [[psi_l(tau*n)] + [0 for _ in range(grid.M - 1)] + [psi_r(tau*n)]]
        return tau, h, u

    @staticmethod
    def solve_by_Lax_Wendroff_scheme(params : HEParams, grid: Grid, phi: RtoR, psi_l: RtoR, psi_r: RtoR):
        tau, h, u = HESolver.__prepare(grid, phi, psi_l, psi_r)
        y = 0
        for n in range(grid.N):
            for m in range(1, grid.M):
                u[n+1][m] = u[n][m]
        return u